# adam [](https://towardsdatascience.com/na%C3%AFve-bayes-from-scratch-using-python-only-no-fancy-frameworks-a1904b37222d)

## paproces

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import re

In [ ]:
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


def hapuskurung(text):
  return re.sub('\[[^]]*\]', '', text)

def hapusurl(text):
  return re.sub(r'http\S+', '', text)

def hapus_spec_cha(text):
  pattern = r'[^a-zA-z0-9\s]'
  text = re.sub(pattern, '', text)
  return text



stops = StopWordRemoverFactory()
stop = stops.get_stop_words()
# print(stop)

#stopword
def hapusstopwordSaswi(text):
  twtext = []
  for i in text.split():
    if i.strip().lower() not in stop and i.strip().lower().isalpha():
      twtext.append(i.strip().lower())
  return " ".join(twtext)

def clean(text):
  text = hapuskurung(text)
  text = hapusurl(text)
  text = hapus_spec_cha(text)
  text = hapusstopwordSaswi(text)
  return text

def preprocess_string(str_arg):
    cleaned_str=re.sub('[^a-z0-9\s]+',' ',str_arg,flags=re.IGNORECASE) #every char except alphabets is replaced
    cleaned_str=re.sub('(\s+)',' ',cleaned_str) #multiple spaces are replaced by single space
    cleaned_str=cleaned_str.lower() #converting the cleaned string to lower case
    
    return cleaned_str # returning the preprocessed string 
    print(cleaned_str)

# df_tweet['text'] = df_tweet['text'].apply(clean)

     |████████████████████████████████| 215kB 2.7MB/s 


## Naive Bayes

In [ ]:
class NaiveBayes:
    
    def __init__(self,unique_classes):
        
        self.classes=unique_classes # Constructor is sinply passed with unique number of classes of the training set
        

    def addToBow(self,example,dict_index):
        
        
        if isinstance(example,np.ndarray): example=example[0]
     
        for token_word in example.split(): #for every word in preprocessed example
          
            self.bow_dicts[dict_index][token_word]+=1 #increment in its count
            
    def train(self,dataset,labels):
        
    
        self.examples=dataset
        self.labels=labels
        self.bow_dicts=np.array([defaultdict(lambda:0) for index in range(self.classes.shape[0])])
        
        #only convert to numpy arrays if initially not passed as numpy arrays - else its a useless recomputation
        
        if not isinstance(self.examples,np.ndarray): self.examples=np.array(self.examples)
        if not isinstance(self.labels,np.ndarray): self.labels=np.array(self.labels)
            
        #constructing BoW for each category
        for cat_index,cat in enumerate(self.classes):
          
            all_cat_examples=self.examples[self.labels==cat] #filter all examples of category == cat
            
            #get examples preprocessed
            
            cleaned_examples=[preprocess_string(cat_example) for cat_example in all_cat_examples]
            
            cleaned_examples=pd.DataFrame(data=cleaned_examples)
            
            #now costruct BoW of this particular category
            np.apply_along_axis(self.addToBow,1,cleaned_examples,cat_index)
      
        prob_classes=np.empty(self.classes.shape[0])
        all_words=[]
        cat_word_counts=np.empty(self.classes.shape[0])
        for cat_index,cat in enumerate(self.classes):
           
            #Calculating prior probability p(c) for each class
            prob_classes[cat_index]=np.sum(self.labels==cat)/float(self.labels.shape[0]) 
            
            #Calculating total counts of all the words of each class 
            count=list(self.bow_dicts[cat_index].values())
            cat_word_counts[cat_index]=np.sum(np.array(list(self.bow_dicts[cat_index].values())))+1 # |v| is remaining to be added
            
            #get all words of this category                                
            all_words+=self.bow_dicts[cat_index].keys()
                                                     
        
        #combine all words of every category & make them unique to get vocabulary -V- of entire training set
        
        self.vocab=np.unique(np.array(all_words))
        self.vocab_length=self.vocab.shape[0]
                                  
        #computing denominator value                                      
        denoms=np.array([cat_word_counts[cat_index]+self.vocab_length+1 for cat_index,cat in enumerate(self.classes)])                                                                          
      
        self.cats_info=[(self.bow_dicts[cat_index],prob_classes[cat_index],denoms[cat_index]) for cat_index,cat in enumerate(self.classes)]                               
        self.cats_info=np.array(self.cats_info)                                 
                                              
                                              
    def getExampleProb(self,test_example):                                                               
                                              
        likelihood_prob=np.zeros(self.classes.shape[0]) #to store probability w.r.t each class
        
        #finding probability w.r.t each class of the given test example
        for cat_index,cat in enumerate(self.classes): 
                             
            for test_token in test_example.split(): #split the test example and get p of each test word
                
                ####################################################################################
                                              
                # for each word w [ count(w|c)+1 ] / [ count(c) + |V| + 1 ]                               
                                              
                ####################################################################################                              
                
                #get total count of this test token from it's respective training dict to get numerator value                           
                test_token_counts=self.cats_info[cat_index][0].get(test_token,0)+1
                
                #now get likelihood of this test_token word                              
                test_token_prob=test_token_counts/float(self.cats_info[cat_index][2])                              
                
                #remember why taking log? To prevent underflow!
                likelihood_prob[cat_index]+=np.log(test_token_prob)
                                              
        # we have likelihood estimate of the given example against every class but we need posterior probility
        post_prob=np.empty(self.classes.shape[0])
        for cat_index,cat in enumerate(self.classes):
            post_prob[cat_index]=likelihood_prob[cat_index]+np.log(self.cats_info[cat_index][1])                                  
      
        return post_prob
    
   
    def test(self,test_set):
       
        predictions=[] #to store prediction of each test example
        for example in test_set: 
                                              
            #preprocess the test example the same way we did for training set exampels                                  
            cleaned_example=preprocess_string(example) 
             
            #simply get the posterior probability of every example                                  
            post_prob=self.getExampleProb(cleaned_example) #get prob of this example for both classes
            
            #simply pick the max value and map against self.classes!
            predictions.append(self.classes[np.argmax(post_prob)])
        
        return np.array(predictions)


In [ ]:
training_set=pd.read_csv('data-tweet-fin-c.csv') # reading the training data-set
type(training_set)
training_set.head()

#getting training set examples labels
y_train=training_set['label'].values
x_train=training_set['text'].values

# tes data set
# from sklearn.model_selection import train_test_split
# train_data,test_data,train_labels,test_labels=train_test_split(x_train,y_train,shuffle=True,test_size=0.01,random_state=42,stratify=y_train)
# classes=np.unique(train_labels)

# Training phase....
nb=NaiveBayes(classes)
nb.train(train_data,train_labels)

# Testing 
testkatanya = [
               "tok inglish mengajarkan istrinya istri tak belajar agama gimana mengajarkan mengajarkan istrinya istri tak belajar agama gimana mengajarkan",
               "aduh bang jago",
               "Allaah bersama orang yang sabar",
               "Ada yang tahu penulisan yang benar itu dimana atau di mana",
               "Tak jarang kata-kata memiliki peran yang cukup besar untuk membangkitkan semangat seseorang ketika sedang merasa tidak baik-baik saja",
               ]
nptestkatanya = np.array(testkatanya)

pclasses=nb.test(nptestkatanya)
# print(test_data)
print(nptestkatanya)
print(pclasses)


for i in range(len(pclasses)):
  if pclasses[i]==0:
    x='tidak terindikasi'
  else:
    x='terindikasi'
  print(nptestkatanya[i], '-', x)

# test_akurasi=np.sum(pclasses==test_labels)/float(test_labels.shape[0])

# print ("Test Set Akurasi: ",test_akurasi) 

['tok inglish mengajarkan istrinya istri tak belajar agama gimana mengajarkan mengajarkan istrinya istri tak belajar agama gimana mengajarkan'
 'aduh bang jago' 'Allaah bersama orang yang sabar'
 'Ada yang tahu penulisan yang benar itu dimana atau di mana'
 'Tak jarang kata-kata memiliki peran yang cukup besar untuk membangkitkan semangat seseorang ketika sedang merasa tidak baik-baik saja']
[0 1 0 0 0]
tok inglish mengajarkan istrinya istri tak belajar agama gimana mengajarkan mengajarkan istrinya istri tak belajar agama gimana mengajarkan - tidak terindikasi
aduh bang jago - terindikasi
Allaah bersama orang yang sabar - tidak terindikasi
Ada yang tahu penulisan yang benar itu dimana atau di mana - tidak terindikasi
Tak jarang kata-kata memiliki peran yang cukup besar untuk membangkitkan semangat seseorang ketika sedang merasa tidak baik-baik saja - tidak terindikasi


In [ ]:
!pip install tweepy
import tweepy as tw

In [ ]:
CONSUMER_key= 'xxxxxxx'
CONSUMER_secret= 'xxxxxxx'
ACCESS_token= 'xxxxxx'
ACCESS_token_secret= 'xxxxxx'

auth = tw.OAuthHandler(CONSUMER_key, CONSUMER_secret)
auth.set_access_token(ACCESS_token, ACCESS_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)



cari_katanya = "hukum buatan manusia"

# tweets = tw.Cursor(api.search,
#               q=search_words,
#               lang="en",
#               since=date_since).items(5)

tweets_katanya = tw.Cursor(api.search,
                           q=cari_katanya,).items(30)

tweets_katanya =[tweet.text for tweet in tweets_katanya] #list
tweets_katanya_c = list(map(clean, tweets_katanya))

print(tweets_katanya)

nptestkatanya = np.array(tweets_katanya_c)
npkatanyaori  = np.array(tweets_katanya)

pclasses=nb.test(nptestkatanya)
# print(test_data)
# print(nptestkatanya)
# print(pclasses)

for i in range(len(pclasses)):
  if pclasses[i]==0:
    x='tidak terindikasi'
  else:
    x='terindikasi'
  print(npkatanyaori[i], '-', x)
print(pclasses)

['RT @AlmayraGucciano: Ketika mata diminta untuk tidak melihat \nKuping tidak untuk mendengar \nMulut tidak untuk bicara \nTangan tidak untuk me…', 'Ketika mata diminta untuk tidak melihat \nKuping tidak untuk mendengar \nMulut tidak untuk bicara \nTangan tidak untuk… https://t.co/HatkKS3Mw3', 'Apa yang bakal dijawab di hari akhir nanti bila ini ditanya Allah al-Hakim?\nHukum buatan manusia memang bebas diuba… https://t.co/vxJuek2Hd0', 'Apa yang bakal dijawab di hari akhir nanti bila ini ditanya Allah al-Hakim?\nHukum buatan manusia memang bebas diuba… https://t.co/PEh02Tibaf', 'RT @hilalludin26gm2: *"Keadilan Hanya pada Hukum Allah"*\n\nMendamba keadilan pada hukum buatan manusia bak mendamba rembulan di siang bolong…', 'Apa yang bakal dijawab di hari akhir nanti bila ini ditanya Allah al-Hakim?\nHukum buatan manusia memang bebas diuba… https://t.co/QVblyVujPc', 'RT @Lub4b1: *"Keadilan Hanya pada Hukum Allah"*\n\nMendamba keadilan pada hukum buatan manusia bak mendamba rembulan di sia